In [1]:
import gc
from pathlib import Path

import h5py

import pandas as pd
import numpy as np

import isic_cnn_multi_predict
import isic_cnn_binary_predict
import isic_boosting_predict
from isic_boosting_predict import pauc_80, PAUC

from isic_helper import time_to_str

In [2]:
id_column = "isic_id"
target_column = "target"
group_column = "patient_id"
fold_column = "gkf_fold"

INPUT_PATH = Path("/kaggle/input/isic-2024-challenge/")
FOLDS_PATH = Path("/kaggle/input/isic-scd-folds")

TOTAL_RUNTIME = 0
TEST_FOLD = -1

EXPECTED_TEST_SIZE = 500000

In [3]:
train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False, na_values=["NA"])
test_metadata = pd.read_csv(INPUT_PATH / "test-metadata.csv", low_memory=False, na_values=["NA"])

folds_df = pd.read_csv(FOLDS_PATH / "folds.csv")
train_metadata = train_metadata.merge(folds_df, on=[id_column, group_column], how="inner")
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

train_images = h5py.File(INPUT_PATH / "train-image.hdf5", mode="r")
test_images = h5py.File(INPUT_PATH / "test-image.hdf5", mode="r")

train_metadata, cat_cols, cont_cols = isic_cnn_binary_predict.cnn_feature_engineering(train_metadata)
test_metadata, _, _ = isic_cnn_binary_predict.cnn_feature_engineering(test_metadata)

Train data size: (401059, 58)
Test data size: (3, 44)


In [4]:
if (TEST_FOLD != -1) and (test_metadata.shape[0] == 3):
    test_metadata = train_metadata[train_metadata[fold_column] == TEST_FOLD].reset_index(drop=True)
    test_images = train_images
    folds_to_run = [TEST_FOLD]
    test_targets = test_metadata[target_column]
else:
    folds_to_run = np.unique(train_metadata[fold_column])
sample_size = test_metadata.shape[0];sample_size

3

In [5]:
cnn_model_names = ["efficientnet_b0", "efficientnet_b1", 
                   "efficientnet_b0", "efficientnet_b1", "efficientnet_b2",
                   "efficientnet_b0", "efficientnet_b1",
                   "tf_efficientnet_b1_ns",
                   "efficientnet_b0"]
cnn_versions = ["v1", "v1", 
                "v1", "v1", "v1",
                "v2", "v2",
                "v1",
                "v3"]
cnn_modes = ["trainmulti", "trainmulti", 
             "trainbinary", "trainbinary", "trainbinary",
             "trainbinary", "trainbinary",
             "trainbinary",
             "trainbinary"]
cnn_paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-{mode}" for model_name, version, mode in zip(cnn_model_names, cnn_versions, cnn_modes)]

all_cnn_oof_columns = []
for idx, path in enumerate(cnn_paths):
    print("\n")
    model_name = cnn_model_names[idx]
    version = cnn_versions[idx]
    mode = cnn_modes[idx]
    
    cnn_oof_train_preds_model_df = pd.read_csv(f"{path}/oof_train_preds_{model_name}_{version}.csv")
    cnn_oof_columns = [col for col in cnn_oof_train_preds_model_df if col.startswith("oof_")]
    cnn_oof_new_columns = [f"{col}_{mode}" for col in cnn_oof_columns]
    columns_mapper = dict(zip(cnn_oof_columns, cnn_oof_new_columns))
    cnn_oof_train_preds_model_df = cnn_oof_train_preds_model_df.rename(columns=columns_mapper)
    all_cnn_oof_columns += cnn_oof_new_columns
    if idx == 0:
        cnn_oof_train_preds_df = cnn_oof_train_preds_model_df[[id_column] + cnn_oof_new_columns].copy()
    else:
        cnn_oof_train_preds_df = cnn_oof_train_preds_df.merge(cnn_oof_train_preds_model_df[[id_column] + cnn_oof_new_columns], on=id_column, how="left")
        assert cnn_oof_train_preds_df.shape[0] == cnn_oof_train_preds_model_df.shape[0]
    
    if mode == "trainmulti":
        cnn_oof_test_preds_model_df, runtime = isic_cnn_multi_predict.run(test_metadata, test_images, model_name, version, path, folds_to_run)
    elif mode == "trainbinary":
        cnn_oof_test_preds_model_df, runtime = isic_cnn_binary_predict.run(test_metadata, test_images, model_name, version, path, folds_to_run, cat_cols, cont_cols)
    cnn_oof_test_preds_model_df = cnn_oof_test_preds_model_df.rename(columns=columns_mapper)
    if idx == 0:
        cnn_oof_test_preds_df = cnn_oof_test_preds_model_df[[id_column] + cnn_oof_new_columns].copy()
    else:
        cnn_oof_test_preds_df = cnn_oof_test_preds_df.merge(cnn_oof_test_preds_model_df[[id_column] + cnn_oof_new_columns], on=id_column, how="left")
    assert cnn_oof_test_preds_df.shape[0] == cnn_oof_test_preds_model_df.shape[0]
    
    TOTAL_RUNTIME += runtime



Predicting for efficientnet_b0_v1
{'debug': False,
 'ext': '2020,2019',
 'fold_column': 'gkf_fold',
 'image_size': 64,
 'init_lr': 3e-05,
 'mixed_precision': 'fp16',
 'mode': 'trainmulti',
 'n_tta': 8,
 'num_epochs': 20,
 'num_workers': 8,
 'seed': 2022,
 'train_batch_size': 64,
 'val_batch_size': 512}

Fold 1
Step: 1/1

Fold 2
Step: 1/1

Fold 3
Step: 1/1

Fold 4
Step: 1/1

Fold 5
Step: 1/1
Time taken: 4.83 s
Predictions generated for efficientnet_b0_v1


Predicting for efficientnet_b1_v1
{'debug': False,
 'ext': '2020,2019',
 'fold_column': 'gkf_fold',
 'image_size': 92,
 'init_lr': 3e-05,
 'mixed_precision': 'fp16',
 'mode': 'trainmulti',
 'n_tta': 8,
 'num_epochs': 20,
 'num_workers': 8,
 'seed': 2022,
 'train_batch_size': 64,
 'val_batch_size': 512}

Fold 1
Step: 1/1

Fold 2
Step: 1/1

Fold 3
Step: 1/1

Fold 4
Step: 1/1

Fold 5
Step: 1/1
Time taken: 5.06 s
Predictions generated for efficientnet_b1_v1


Predicting for efficientnet_b0_v1
{'debug': False,
 'down_sampling': True,
 'f

In [6]:
del train_metadata, test_metadata
gc.collect()

73083

In [7]:
train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False, na_values=["NA"])
test_metadata = pd.read_csv(INPUT_PATH / "test-metadata.csv", low_memory=False, na_values=["NA"])

folds_df = pd.read_csv(FOLDS_PATH / "folds.csv")
train_metadata = train_metadata.merge(folds_df, on=[id_column, group_column], how="inner")
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

train_metadata, _ = isic_boosting_predict.boosting_feature_engineering(train_metadata)
test_metadata, _ = isic_boosting_predict.boosting_feature_engineering(test_metadata)

if len(all_cnn_oof_columns) > 0:
    train_metadata = train_metadata.merge(cnn_oof_train_preds_df, on=id_column, how="left")
    test_metadata = test_metadata.merge(cnn_oof_test_preds_df, on=id_column, how="left")

Train data size: (401059, 58)
Test data size: (3, 44)


In [8]:
if (TEST_FOLD != -1) and (test_metadata.shape[0] == 3):
    test_metadata = train_metadata[train_metadata[fold_column] == TEST_FOLD].reset_index(drop=True)
sample_size = test_metadata.shape[0];sample_size

3

In [9]:
boosting_model_names = ["xgb", "xgb", "xgb", "lgb", "cb"]
boosting_versions = ["v1", "v3", "v5", "v6", "v1"]
boosting_modes = ["train", "train", "train", "train", "train"]
boosting_paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-{mode}" for model_name, version, mode in zip(
    boosting_model_names, boosting_versions, boosting_modes)]

all_boosting_oof_columns = []
for idx, path in enumerate(boosting_paths):
    print("\n")
    model_name = boosting_model_names[idx]
    version = boosting_versions[idx]
    mode = boosting_modes[idx]
    
    boosting_oof_train_preds_model_df = pd.read_csv(f"{path}/oof_train_preds_{model_name}_{version}.csv")
    boosting_oof_columns = [col for col in boosting_oof_train_preds_model_df if col.startswith("oof_")]
    all_boosting_oof_columns += boosting_oof_columns
    if idx == 0:
        boosting_oof_train_preds_df = boosting_oof_train_preds_model_df[[id_column] + boosting_oof_columns].copy()
    else:
        boosting_oof_train_preds_df = boosting_oof_train_preds_df.merge(boosting_oof_train_preds_model_df[[id_column] + boosting_oof_columns], on=id_column, how="left")
        assert boosting_oof_train_preds_df.shape[0] == boosting_oof_train_preds_df.shape[0]
        
    boosting_oof_test_preds_model_df, runtime = isic_boosting_predict.run(train_metadata, test_metadata, model_name, version, path, folds_to_run)
    if idx == 0:
        boosting_oof_test_preds_df = boosting_oof_test_preds_model_df[[id_column] + boosting_oof_columns].copy()
    else:
        boosting_oof_test_preds_df = boosting_oof_test_preds_df.merge(boosting_oof_test_preds_model_df[[id_column] + boosting_oof_columns], on=id_column, how="left")
    assert boosting_oof_test_preds_df.shape[0] == boosting_oof_test_preds_df.shape[0]
    
    TOTAL_RUNTIME += runtime



Predicting for xgb_v1
{'best_num_rounds': {'fold_1': 918,
                     'fold_2': 245,
                     'fold_3': 1668,
                     'fold_4': 362,
                     'fold_5': 651},
 'config': {'_key': None,
            '_parent': None,
            '_temp': False,
            'fold_column': 'gkf_fold',
            'model_name': 'xgb_v1',
            'models_output_dir': 'models',
            'sampling_ratio': 0.01,
            'seed': 2022},
 'cv_auc_avg': 0.9783121162985614,
 'cv_auc_oof': 0.9325410500676679,
 'cv_auc_std': 0.008192836963125776,
 'cv_pauc_avg': 0.1820560788195227,
 'cv_pauc_oof': 0.13681816531200738,
 'cv_pauc_std': 0.007177074219698841,
 'es_rounds': 250,
 'num_rounds': 3000,
 'params': {'alpha': 0.6779926606782505,
            'colsample_bylevel': 0.5476090898823716,
            'colsample_bynode': 0.9928601203635129,
            'colsample_bytree': 0.8437772277074493,
            'disable_default_eval_metric': True,
            'enable_categ

In [10]:
if (TEST_FOLD != -1):
    factor = EXPECTED_TEST_SIZE / sample_size
    expected_total_runtime = TOTAL_RUNTIME * factor * 5
    print(f"Expected total runtime during submission: {time_to_str(expected_total_runtime)}")

In [11]:
ensemble_preds_df = boosting_oof_test_preds_df.merge(cnn_oof_test_preds_df, on=id_column, how="left")

weights = {'oof_cb_v1': 0.9321011424471168,
 'oof_efficientnet_b0_v2_trainbinary': 4.34378768545616,
 'oof_efficientnet_b0_v3_trainbinary': 1.6648020726137487,
 'oof_efficientnet_b1_v1_trainbinary': 1.0853522706879721,
 'oof_efficientnet_b1_v2_trainbinary': 6.030000016900447,
 'oof_lgb_v6': 7.65238938893904,
 'oof_tf_efficientnet_b1_ns_v1_trainbinary': 4.083353259249208,
 'oof_xgb_v1': 5.184454122392706,
 'oof_xgb_v3': 10.0,
 'oof_xgb_v5': 4.979458848219839}

ensemble_preds = 0
for oof_column, weight in weights.items():
    ensemble_preds += ensemble_preds_df[oof_column].rank(pct=True).values * weight
ensemble_preds_df[target_column] = ensemble_preds
ensemble_preds_df.head()

,isic_id,oof_xgb_v1,oof_xgb_v3,oof_xgb_v5,oof_lgb_v6,oof_cb_v1,oof_efficientnet_b0_v1_trainmulti,oof_efficientnet_b0_v1_AKIEC_trainmulti,oof_efficientnet_b0_v1_BCC_trainmulti,oof_efficientnet_b0_v1_BKL_trainmulti,...,oof_efficientnet_b1_v1_VASC_trainmulti,oof_efficientnet_b1_v1_unknown_trainmulti,oof_efficientnet_b0_v1_trainbinary,oof_efficientnet_b1_v1_trainbinary,oof_efficientnet_b2_v1_trainbinary,oof_efficientnet_b0_v2_trainbinary,oof_efficientnet_b1_v2_trainbinary,oof_tf_efficientnet_b1_ns_v1_trainbinary,oof_efficientnet_b0_v3_trainbinary,target
0,ISIC_0015657,0.011152,0.355757,0.311791,0.190844,0.054264,0.001234,0.000142,0.000799,0.000456,...,5.414601e-07,0.999760,0.002732,0.000534,0.002771,0.001824,0.000516,0.001158,0.001700,40.962546
1,ISIC_0015729,0.004999,0.203587,0.253710,0.065381,0.002919,0.000025,0.000002,0.000015,0.000028,...,1.772390e-07,0.999970,0.001435,0.000102,0.000247,0.000073,0.000025,0.000018,0.000252,19.872226
2,ISIC_0015740,0.005263,0.380009,0.366158,0.047489,0.005606,0.000034,0.000003,0.000026,0.000008,...,3.349879e-07,0.999949,0.001809,0.000159,0.000853,0.000056,0.000049,0.000095,0.000119,31.076626


In [12]:
ensemble_preds_df[target_column].describe()

count     3.000000
mean     30.637133
std      10.552027
min      19.872226
25%      25.474426
50%      31.076626
75%      36.019586
max      40.962546
Name: target, dtype: float64

In [13]:
ensemble_preds_df[[id_column, target_column]].head()

,isic_id,target
0,ISIC_0015657,40.962546
1,ISIC_0015729,19.872226
2,ISIC_0015740,31.076626


In [14]:
ensemble_preds_df[[id_column, target_column]].to_csv("submission.csv", index=False)

In [15]:
# from isic_helper import compute_auc, compute_pauc
# testing_pauc = compute_pauc(test_targets, ensemble_preds_df[target_column], min_tpr=0.8)
# testing_auc = compute_auc(test_targets, ensemble_preds_df[target_column])
# testing_auc, testing_pauc

In [16]:
# Expected total runtime during submission: 543 min 42.76 sec
# Fold: 3 (0.9758723215734236, 0.17990228500829888)